# Problem Formulation
Now a days the world has many social problems, one of those is fake information. we are going to predict if news are fakeor real by lookinh at it's tittle.                                                                                      
data contains raw (different forms of words) so it is up to you on what text preprocessing techniques to be applied

# The input and output
The input contains new row after preprocessing (new_text)                         
The output represents as y (label feature)

# What data mining function is required?
predict function is the process of identifying the unavailable data for new observation based on the previous data and classification function to identify class label of new observation to which it belongs.

# What could be the challenges?  
The training data given has y_label as 0,1 or 2. The label "2" is not the accurate labels as the final output can be 0 or 1 only. I removed those entries with label as 2
-how to handle upper case in words with lower case                                                                     
-remove stop words                                                                   
-make stemming process                                   
-remove any html tags                            
-remove single letter chars                                                       
-convert all whitespaces to single whitespace

# Why do we remove stop words? 
stop words are available in abundance in any human language. By removing these words, we remove the low-level information from our text in order to give more focus to the important information.
Removal of stop words definitely reduces the dataset size and thus reduces the training time due to the fewer number of tokens involved in the training.

# why do we make stemmer?
 is a technique used to reduce text dimensionality. Stemming is also a type of text normalization that enables you to standardize some words into specific expressions also called stems

# What is the impact?
analyze the performance of the models when trained on each resulting dataset with respect to f1-score and roc scoring .       
After preprocessing for data , i apply classifier models to predict if the news are fake or real.
### First classifier is logistic regression 
with word vectorizer that has F1-score=84 is larger than logistic regression with char vectorizer that has f1-score=69.

### The second one is XGboost
That has f1-score=0.81 with word vectorizer
second i increased the values of hyperparameters. That has f1-score=0.83 with word vectorizer
Third with char vectorizer ,it gives f1-score= 0.71
### Third one is random forest classifier
that has 1-score =81 
### The fourth one is KNN classifier 
that has f1-score =77 
### Fifth is decision tree 
with word vectorizer has f1-score=67 and 
with char vectorizer has f1-score =65
#### As we show the result with character vectorizer is not the best choise but word vectorizer gives larger f1-score than char vectorizer 

# What is an ideal solution?
ideal best solution is logistic regression classifier , it gives high F1-score with word vectorizer =0.84

# importing libraries

In [1]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import nltk #natural lan tool kit
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from skopt import BayesSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import PredefinedSplit
from sklearn.tree import DecisionTreeClassifier


# read data
The data in this problem is in the text format. Each data entry contains number of words which represents the title of the news. It is a labelled data with label as 1 or 0 which implies if the news is fake or not respectively. Number of words in each data entry varies as the title length changes. So the input to the model is the text data 'text of the news' and output of the model is the prediction whether the give titled news is fake or not

In [2]:
data_train=pd.read_csv('xy_train.csv',index_col='id')
data_test=pd.read_csv('x_test.csv')


# Data exploration


In [3]:
data_train.isnull().sum() #check null value for every feature

text     0
label    0
dtype: int64

In [4]:
d=data_train.loc[data_train["label"] >= 2] # display how many row with label=2

In [5]:
d.value_counts

<bound method DataFrame.value_counts of                                                      text  label
id                                                              
540454  World's Funniest Joke|鈥淭wo hunters are out in ...      2
342238  The Simpsons writer: 'Why I predicted Donald T...      2
552146  Kanye's philosophy book could explain why he d...      2
549212  Larry David Gave Rob McElhenney This Advice Ab...      2
398378  Parents Who Bring Forgotten Lunch to School Se...      2
...                                                   ...    ...
219497  鈥淭hree tips on getting your beach body that do...      2
54937   Disgusting: ISIS Just Released A 2-Star Review...      2
505566  I Used To Think All Terrorists Were Muslim. Th...      2
288391  He鈥檚 Running: Joe Biden Just Made A Pinterest ...      2
99749   John Oliver rips Jeff Sessions and names Trump...      2

[232 rows x 2 columns]>

In [6]:
# clean noise (label =2) 
data_train.drop(data_train.index[data_train['label']==2],inplace=True)

## Cleaning and pre processing
NLP deals with text . we must remove html tags ,single letter charachters , stopwords ,punctutaion and make stemmer.

In [7]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))
#make function to do the process of removing html and single letter charachters ,stopwords ,punctutaion and make stemmer.

def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE) # to remove white space
    RE_TAGS = re.compile(r"<[^>]+>") # remove html tags
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE) 
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE) # to remove single charachter
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text) # replace with white space
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to C:\Users\Lab
[nltk_data]     3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Lab
[nltk_data]     3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
data_train["new_text"] = data_train.loc[data_train["text"].str.len() > 0, "text"]
data_train["new_text"] = data_train["new_text"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
data_test["new_text"] = data_test.loc[data_test["text"].str.len() > 0, "text"]
data_test["new_text"] = data_test["new_text"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)


In [9]:
#define x and y 
x=data_train["new_text"]
y=data_train["label"]
x

id
265723    group of friend began to volunte at homeless s...
284269    british prim minist theresa may on nerv attack...
207715    goodyear released kit that allows ps to be bro...
551106    happy birthday bob bark the pric is right host...
8584      obama to nation innocent cops and unarmed youn...
                                ...                        
70046     finish snip simo yh during the invasion of fin...
189377    nigerian princ scam took from kansas year lat ...
93486     is it saf to smok marijuana during pregnancy y...
140950    julius caesar upon realizing that everyon the ...
34509     jeff bridg releasing leeping tap new album des...
Name: new_text, Length: 59768, dtype: object

# split data with 80% training and 20% testing

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

## make two pipelines contain char and word vecrorizer 
tokenization is made fistly in which to split the sentence into words then calculate the frequency for each word.
for the Tf is Term frequency, and IDF is Inverse document frequency. TF-IDF is a method which gives us a numerical weightage of words which reflects how important the particular word is to a document in a corpus. A corpus is a collection of documents.
The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams or single words for example, “Medium blog” is a 2-gram (a bigram), (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. Only applies if analyzer is not callable.

In [11]:
#first pipeline for word vectorizer
pipe_vect1 = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
))])
#first pipeline for char vectorizer

pipe_vect2=Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer="char", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
))])




pipe_vect2.fit(x_train,y_train)
pipe_vect1.fit(x_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.3, min_df=10, ngram_range=(1, 2)))])

In [12]:
data_testd=data_test.drop(columns=['id'])
data_testd

,text,new_text
0,stargazer,stargaz
1,yeah,yeah
2,PD: Phoenix car thief gets instructions from Y...,pd phoenix car thief get instruction from yout...
3,"As Trump Accuses Iran, He Has One Problem: His...",as trump accus iran he has one probl his own c...
4,"""Believers"" - Hezbollah 2011",believ hezbollah
...,...,...
59146,Bicycle taxi drivers of New Delhi,bicycl taxi driv of new delhi
59147,Trump blows up GOP's formula for winning House...,trump blows up gop formula for winning hous rac
59148,Napoleon returns from his exile on the island ...,napoleon return from his exil on the island of...
59149,Deep down he always wanted to be a ballet dancer,deep down he always wanted to be ballet danc


In [13]:
test_=data_testd['new_text']
test_ # select row that has new text

0                                                  stargaz
1                                                     yeah
2        pd phoenix car thief get instruction from yout...
3        as trump accus iran he has one probl his own c...
4                                         believ hezbollah
                               ...                        
59146                        bicycl taxi driv of new delhi
59147      trump blows up gop formula for winning hous rac
59148    napoleon return from his exil on the island of...
59149         deep down he always wanted to be ballet danc
59150    toddl miraculously surviv story fall landing o...
Name: new_text, Length: 59151, dtype: object

# Grid search with validation set

In [14]:
x_train2, x_val, y_train2, y_val = train_test_split(
    x_train, y_train, train_size = 0.8, stratify = y_train, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in x_train2.index else 0 for x in x_train.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

### logistic regression
# Trial 1
#### i will make logistic regression with word vectorizer

In [15]:
pipe=Pipeline([
    ('preprocessor',pipe_vect1),
    ('classifier', LogisticRegression()),
])
C = [1.0, 0.1, 0.01]
penalty = ['l1', 'l2']

param_grid = {
                  'classifier__C': C,
                  'classifier__penalty': penalty,
             'classifier__solver':['lbfgs', 'liblinear', 'sag']}

# cv=2 means two-fold cross-validation
# n_jobs means the cucurrent number of jobs
# (on colab since we only have two cpu cores, we set it to 2)
grid_search = GridSearchCV(
    pipe, param_grid, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X_train; but the grid search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set
grid_search.fit(x_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

Fitting 1 folds for each of 18 candidates, totalling 18 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan 0.87643914        nan 0.88475816 0.88475732 0.8847571
        nan 0.79238124        nan 0.862386   0.86238807 0.86238076
        nan 0.53753637        nan 0.83682569 0.83693395 0.83682384]
  warnings.warn(


best score 0.8847581594878194
best score {'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}


In [16]:
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['label'] =grid_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissiondec2_walkthrough.csv', index=False)

### Logistic regression model gives f1-score=0.84928 in leaderboard

# Trial 2
#### i will change word vectorizer by char vectorizer with diff hyperparameter

In [17]:
# firstly, make pipeline contains vectorizer and classifier
pipe=Pipeline([
    ('preprocessor',pipe_vect2),
    ('classifier', LogisticRegression()),
])
C = [1.0, 10, 100, 1e3,1e5]
penalty = ['l1', 'l2']

param_grid = {
                  'classifier__C': C,
                  'classifier__penalty': penalty,
             'classifier__solver':['newton-cg', 'lbfgs', 'liblinear', 'saga']}

# n_jobs means the cucurrent number of jobs
# (on colab since we only have two cpu cores, we set it to 2)
grid_search = GridSearchCV(
    pipe, param_grid, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

# here we still use X_train; but the grid search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set
grid_search.fit(x_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

Fitting 1 folds for each of 40 candidates, totalling 40 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.7485954  0.7485921  0.74809574 0.74810075
 0.74809134 0.74809728        nan        nan 0.74705803 0.74708712
 0.74758828 0.74757442 0.7475885  0.74758723        nan        nan
 0.7459945  0.74628367 0.74645504 0.74659362 0.74645596 0.74647053
        nan        nan 0.74572645 0.74621383 0.74596602 0.74595889
 0.74597491 0.74623584        nan        nan 0.74556362 0.74620904
 0.74539894 0.74616934 0.74549787 0.7462071 ]
  warnings.warn(


best score 0.7485954027682672
best score {'classifier__C': 1.0, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}


In [18]:
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['label'] =grid_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissiondeclog char_walkthrough.csv', index=False)

### as i show ,when i used char vectorizer and make grid search ,the result is f1-score= 0.69924 which mean the word vectorizer is better than char vectorizer



## Trail 3 for XGBclassifier
i try to use different classifier to see what happen 
-XGBclassifier with word vectorizer

In [42]:
# make pipeline with word vectorizer and classifier
full_pipline = Pipeline([
    ('preprocessor',pipe_vect1),
    ('classifier', XGBClassifier()),
])        
#make bayes search
bayes_search = BayesSearchCV(
    full_pipline,
    {
     'classifier__max_depth':[1,4,5,7,8],
    'classifier__n_estimators': [100, 200,300, 400,500,600],
    'classifier__learning_rate': [0.1, 0.01, 0.05]}
,
    # number of trials 
    n_iter=3,
    random_state=0,
    verbose=1,
    # we still use 
    cv=pds, scoring='roc_auc'
)

bayes_search.fit(x_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:09:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:10:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:11:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:11:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.8586988695274089
best score OrderedDict([('classifier__learning_rate', 0.05), ('classifier__max_depth', 8), ('classifier__n_estimators', 600)])


In [43]:
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['label'] =bayes_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissiondecxg_walkthrough.csv', index=False)

### f1-score =0.81899 ,it's mean that the logistic regression with word vectorizer has more f1-score than XGBoost

# Trial 4
## Xgboost with different values of hyperparameters
i will try to give largest values for hyperparameters 

In [21]:
# pipeline contains word vectorizer and classifier

full_pipline = Pipeline([
    ('preprocessor',pipe_vect1),
    ('classifier', XGBClassifier()),
])        
bayes_search = BayesSearchCV(
    full_pipline,
    {
     'classifier__max_depth':[10,12,13,14],
    'classifier__n_estimators': [700,800,900],
    'classifier__learning_rate': [0.1, 0.01, 0.05,0.02]}
,
    # number of trials 
    n_iter=2,
    random_state=42,
    verbose=1,
    # we still use 
    cv=pds, scoring='roc_auc'
)

bayes_search.fit(x_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:00:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:08:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.1min finished
C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:15:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.8752546511853063
best score OrderedDict([('classifier__learning_rate', 0.1), ('classifier__max_depth', 14), ('classifier__n_estimators', 800)])


In [22]:
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['label'] =bayes_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissionxgb2_walkthrough.csv', index=False)

### f1-score has 0.83002 ,which mean that large values for hyperparameter are given better performance than small values 

# Trial 5
## Xgboost with char vectorizer 

In [19]:
# pipeline contains char vectorizer and classifier
full_pipline = Pipeline([
    ('preprocessor',pipe_vect2),
    ('classifier', XGBClassifier()),
])        
bayes_search = BayesSearchCV(
    full_pipline,
    {
     'classifier__max_depth':[10,12,13,14],
    'classifier__n_estimators': [700,800,900],
    'classifier__learning_rate': [0.1, 0.01, 0.05,0.02]}
,
    # number of trials 
    n_iter=2,
    random_state=42,
    verbose=1,
    # we still use 
    cv=pds, scoring='roc_auc'
)

bayes_search.fit(x_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:55:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:58:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:01:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.7772293255715295
best score OrderedDict([('classifier__learning_rate', 0.1), ('classifier__max_depth', 14), ('classifier__n_estimators', 800)])


In [20]:
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['label'] =bayes_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissionxgb2_walkthrough.csv', index=False)

## when i use char vectorizer the f1-score is decreased to 0.71939

# Trial 6
## RandomCLassifier
i will try diff classifier to see what is the best

In [25]:
# pipeline contains word vectorizer and classifier
pipe=Pipeline([
    ('preprocessor',pipe_vect1),
    ('my_classifierran', 
           RandomForestClassifier(random_state=42,class_weight = 'balanced'),)
])        
random_grid = {
    'my_classifierran__max_features': ['auto', 'sqrt'],
    'my_classifierran__n_estimators': [70,80,100,300],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'my_classifierran__max_depth':[10, 20, 30, 40, 50, 60], 
    'my_classifierran__min_samples_split': [2, 5, 10], 
    'my_classifierran__min_samples_leaf': [1, 2, 4]
}

# cv=2 means two-fold cross-validation
# n_jobs means the cucurrent number of jobs
# (on colab since we only have two cpu cores, we set it to 2)
random_search = GridSearchCV(
    pipe, random_grid, cv=pds, verbose=1, n_jobs=3, 
    scoring='roc_auc')

random_search.fit(x_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))

Fitting 1 folds for each of 432 candidates, totalling 432 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.6min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 12.8min
[Parallel(n_jobs=3)]: Done 432 out of 432 | elapsed: 42.2min finished


best score 0.8507153249272552
best score {'my_classifierran__max_depth': 60, 'my_classifierran__max_features': 'auto', 'my_classifierran__min_samples_leaf': 1, 'my_classifierran__min_samples_split': 2, 'my_classifierran__n_estimators': 300}


In [26]:
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['label'] =random_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissionrandom_walkthrough.csv', index=False)

### f1-score =0.81594 but still logistic regression is the best

# Trial 7
## KNN classifier
### try to make KNN with word vectorizer

In [23]:
pipe=Pipeline([('preprocessor',pipe_vect1),
        ('my_classifier',KNeighborsClassifier())])
k_range=list(range(1,30))
#Creating parameter grid
weights = ['uniform', 'distance']
param={'my_classifier__n_neighbors':k_range
       ,'my_classifier__weights':weights}
search=BayesSearchCV(pipe,param,cv=pds,scoring='roc_auc')
#fit model
search.fit(x_train, y_train)

print('best score {}'.format(search.best_score_))
print('best score {}'.format(search.best_params_))

C:\Users\Lenovo\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lenovo\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lenovo\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lenovo\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lenovo\anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\Lenovo

best score 0.8270945516572459
best score OrderedDict([('my_classifier__n_neighbors', 29), ('my_classifier__weights', 'distance')])


In [24]:
#make file submission
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['label'] =search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissionknn_walkthrough.csv', index=False)

### f1-score =0.77568 

# Trial 8

## Decision tree

In [24]:
# pipeline with word vectorizer
pipe=Pipeline([('preprocessor',pipe_vect1),
        ('my_classifier',DecisionTreeClassifier())])
decision_search={
    "my_classifier__criterion" : ['gini','entropy'],
     "my_classifier__max_depth" : [10, 12,13,14],
     "my_classifier__min_samples_leaf": [100,200,300,500,600,800]
 }
#make bayessearch
decision_search = BayesSearchCV(
    pipe, decision_search, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')
# fit model
decision_search.fit(x_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))
#make file submission
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['label'] =decision_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissiondec_walkthrough.csv', index=False)

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fi

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits


...

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score 0.70488240233347
best score OrderedDict([('my_classifier__criterion', 'gini'), ('my_classifier__max_depth', 14), ('my_classifier__min_samples_leaf', 100)])


### f1-score =0.67215 , decision tree is the worst model

# Trial 9
## decision tree with char vectorizer
i try to use char vectorizer to see if this model keeps the worst.

In [21]:
pipe=Pipeline([('preprocessor',pipe_vect2),
        ('my_classifier',DecisionTreeClassifier())])
decision_search={
    "my_classifier__criterion" : ['gini','entropy'],
     "my_classifier__max_depth" : [10, 12,13,14],
     "my_classifier__min_samples_leaf": [100,200,300,500,600,800]
 }
decision_search = BayesSearchCV(
    pipe, decision_search, cv=pds, verbose=1, n_jobs=2, 
    scoring='roc_auc')

decision_search.fit(x_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fi

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


...

Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score 0.6791013297399491
best score OrderedDict([('my_classifier__criterion', 'entropy'), ('my_classifier__max_depth', 13), ('my_classifier__min_samples_leaf', 100)])


In [22]:
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['label'] =decision_search.predict_proba(test_)[:,1]
submission.to_csv('sample_submissiondecchar_walkthrough.csv', index=False)

### f1-score =0.65622 , decision tree is the worst model

# What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?


-Represent unique word sequence of length n as feature                                     
-Represent unique character sequence of length n as feature                                         
-Character n-gram tends to suffer more from the OOV issue ,if a OOV word is present
ten times in the corpus, it is counted ten times in the
character n-gram). Note that it is also possible to include
more data from other sources to enrich the character-level
language model

# What is the difference between stop word removal and stemming? Are these techniques language-dependent?
-stopword removal is removing the words that occur commonly across all the documents. Typically, articles and pronouns are generally classified as stop words
-stemming is the process of removing a part of a word, or reducing a word to its stem or root.            
-stemming and stopword are dependent

# Is tokenization techniques language dependent? Why?
Yes it's dependent because tokenization is spliting the raw text into small token or single word. These tokens help in understanding the context or developing the model for the NLP and helps in interpreting the meaning of the text by analyzing the sequence of the words.

## What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?
tf-idf vectorizer returns floats, not only focuses on the frequency of words present in the corpus but also provides the importance of the words,while the count vectorizer returns integers ,converts a given set of strings into a frequency representation .
No, because the best N-grams are chosen by tuning and testing the previous n-grams extraction query on different hyperparameter values, including different sentences examples and N-gram size